Play around with parsing data from AVIRIS Fusion Table (converted to csv).

TODO convert to GeoPandas.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
flights = pd.read_csv('Filtered HyspIRI Flights.csv')
# It's possible to use a column as row labels if desired.
# flights = pd.read_csv('Filtered HyspIRI Flights.csv', index_col=0)
flights.info(verbose=False)
flights.head()

In [ ]:
# In case you care.
# flights.describe()
# flights.count()

In [ ]:
# Select years.
years = [2013, 2014, 2015]
flights = flights[flights.Year.isin(years)]  # Note isin() returns a Series.

# Equivalent way to do the same, to illustrate Pandas synatax. Note implied : for columns when using loc/iloc.
assert flights.equals(flights[lambda df: df.Year.isin(years)])
assert flights.equals(flights.loc[flights.Year.isin(years)])
assert flights.equals(flights.loc[lambda df: df.Year.isin(years)])
assert flights.equals(flights.iloc[lambda df: df.Year.isin(years).values])  # Must extract values, array() or to_numpy() also work.

flights.info(verbose=False)
sorted(flights.Year.unique())

In [ ]:
# Select sites.
pattern = 'Yosemite'  # Try 'Yosemite|Tahoe'.
flights = flights[flights.site_name.str.contains(pattern, na=False)]  # Treat NaN values as

flights.info(verbose=False)
sorted(flights.site_name.unique())
# An alternative way. Since sort_values() returns a Series use series.unique(), which returns an ndarray.
# flights3.site_name.sort_values().unique()

In [ ]:
# TODO use GeoPandas!

from shapely.geometry import Polygon, MultiPolygon

# Select subset of data.
f = flights

# Create Shapely objects.
polygons = []
for index, row in f.iterrows():
    a = (row.Lon1, row.Lat1)
    b = (row.Lon2, row.Lat2)
    c = (row.Lon3, row.Lat3)
    d = (row.Lon4, row.Lat4)

    # Construct shapely Polygon using WGS 84 bounds.
    polygon = Polygon((a, b, c, d))
    polygons.append(polygon)

multipolygon = MultiPolygon(polygons)
multipolygon

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()
ax.stock_img()

# Among other things it's possible to add a MultiPolygon object or a list of Polygons.
ax.add_geometries(multipolygon, ccrs.PlateCarree(), edgecolor='black', facecolor='red', alpha=0.8)

bounds = multipolygon.bounds
padding = 2
# ax.set_extent((bounds[0]-padding, bounds[2]+padding, bounds[1]-padding, bounds[3]+padding), ccrs.Geodetic())
ax.set_extent((bounds[0]-padding, bounds[2]+padding, bounds[1]-padding, bounds[3]+padding), ccrs.Geodetic())
# plt.show()
